# Loop 42 Strategic Analysis

**Goal**: Decide whether to submit exp_043 or try something else first.

**Key Questions**:
1. What is the expected value of submitting exp_043?
2. Are there other approaches that might be more promising?
3. How should we allocate our remaining 5 submissions?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
]

df = pd.DataFrame(submissions)
print('Submission History:')
print(df)
print(f'\nBest CV: {df["cv"].min():.4f} ({df.loc[df["cv"].idxmin(), "exp"]})')
print(f'Best LB: {df["lb"].min():.4f} ({df.loc[df["lb"].idxmin(), "exp"]})')

Submission History:
        exp      cv      lb
0   exp_000  0.0111  0.0982
1   exp_001  0.0123  0.1065
2   exp_003  0.0105  0.0972
3   exp_005  0.0104  0.0969
4   exp_006  0.0097  0.0946
5   exp_007  0.0093  0.0932
6   exp_009  0.0092  0.0936
7   exp_012  0.0090  0.0913
8   exp_024  0.0087  0.0893
9   exp_026  0.0085  0.0887
10  exp_030  0.0083  0.0877

Best CV: 0.0083 (exp_030)
Best LB: 0.0877 (exp_030)


In [2]:
# Fit linear relationship between CV and LB
cv_vals = df['cv'].values
lb_vals = df['lb'].values

slope, intercept, r_value, p_value, std_err = stats.linregress(cv_vals, lb_vals)

print(f'CV-LB Relationship: LB = {slope:.2f} x CV + {intercept:.4f}')
print(f'R-squared = {r_value**2:.4f}')
print(f'Standard error of slope: {std_err:.4f}')

# Calculate residuals
predicted_lb = slope * cv_vals + intercept
residuals = lb_vals - predicted_lb
print(f'\nResidual statistics:')
print(f'  Mean: {np.mean(residuals):.6f}')
print(f'  Std: {np.std(residuals):.6f}')
print(f'  Max: {np.max(residuals):.6f}')
print(f'  Min: {np.min(residuals):.6f}')

CV-LB Relationship: LB = 4.30 x CV + 0.0524
R-squared = 0.9675
Standard error of slope: 0.2629

Residual statistics:
  Mean: -0.000000
  Std: 0.000925
  Max: 0.001579
  Min: -0.001999


In [3]:
# What would it take to reach the target?
target = 0.0347

print(f'=== REACHING THE TARGET ===')
print(f'Target: {target}')
print(f'Current intercept: {intercept:.4f}')
print(f'Gap: {intercept - target:.4f}')

# If we could reduce the intercept to 0.02, what CV would we need?
for new_intercept in [0.04, 0.03, 0.02, 0.01, 0.00]:
    required_cv = (target - new_intercept) / slope
    print(f'\nIf intercept = {new_intercept:.2f}:')
    print(f'  Required CV: {required_cv:.4f}')
    if required_cv > 0:
        print(f'  This is achievable!')
    else:
        print(f'  This is impossible (negative CV)')

=== REACHING THE TARGET ===
Target: 0.0347
Current intercept: 0.0524
Gap: 0.0177

If intercept = 0.04:
  Required CV: -0.0012
  This is impossible (negative CV)

If intercept = 0.03:
  Required CV: 0.0011
  This is achievable!

If intercept = 0.02:
  Required CV: 0.0034
  This is achievable!

If intercept = 0.01:
  Required CV: 0.0057
  This is achievable!

If intercept = 0.00:
  Required CV: 0.0081
  This is achievable!


In [4]:
# Analyze exp_043 (aggressive regularization)
exp_043_cv = 0.009002

# Predicted LB using old relationship
predicted_lb_043 = slope * exp_043_cv + intercept

print(f'=== exp_043 ANALYSIS ===')
print(f'CV: {exp_043_cv:.6f}')
print(f'Predicted LB (using old relationship): {predicted_lb_043:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: {target}')

# What if aggressive regularization changes the relationship?
print(f'\n=== HYPOTHESIS TESTING ===')
print(f'If actual LB < {predicted_lb_043:.4f}: Overfitting hypothesis CONFIRMED')
print(f'If actual LB < 0.0877: Regularization HELPS')
print(f'If actual LB > {predicted_lb_043:.4f}: Hypothesis REJECTED')

# What would the new relationship need to be to reach target?
print(f'\n=== WHAT WE NEED ===')
print(f'To reach target {target} with CV {exp_043_cv:.4f}:')
required_intercept = target - slope * exp_043_cv
print(f'  Required intercept: {required_intercept:.4f}')
print(f'  Current intercept: {intercept:.4f}')
print(f'  Reduction needed: {intercept - required_intercept:.4f}')

=== exp_043 ANALYSIS ===
CV: 0.009002
Predicted LB (using old relationship): 0.0912
Best LB so far: 0.0877
Target: 0.0347

=== HYPOTHESIS TESTING ===
If actual LB < 0.0912: Overfitting hypothesis CONFIRMED
If actual LB < 0.0877: Regularization HELPS
If actual LB > 0.0912: Hypothesis REJECTED

=== WHAT WE NEED ===
To reach target 0.0347 with CV 0.0090:
  Required intercept: -0.0040
  Current intercept: 0.0524
  Reduction needed: 0.0565


In [5]:
# Calculate CV-LB ratios
df['cv_lb_ratio'] = df['lb'] / df['cv']

print('CV-LB Ratios:')
print(df[['exp', 'cv', 'lb', 'cv_lb_ratio']].to_string(index=False))
print(f'\nMean ratio: {df["cv_lb_ratio"].mean():.2f}x')
print(f'Min ratio: {df["cv_lb_ratio"].min():.2f}x ({df.loc[df["cv_lb_ratio"].idxmin(), "exp"]})')
print(f'Max ratio: {df["cv_lb_ratio"].max():.2f}x ({df.loc[df["cv_lb_ratio"].idxmax(), "exp"]})')

# Is the ratio increasing as CV improves?
print(f'\n=== RATIO TREND ===')
for i in range(len(df)):
    print(f'{df.iloc[i]["exp"]}: CV {df.iloc[i]["cv"]:.4f}, Ratio {df.iloc[i]["cv_lb_ratio"]:.2f}x')

CV-LB Ratios:
    exp     cv     lb  cv_lb_ratio
exp_000 0.0111 0.0982     8.846847
exp_001 0.0123 0.1065     8.658537
exp_003 0.0105 0.0972     9.257143
exp_005 0.0104 0.0969     9.317308
exp_006 0.0097 0.0946     9.752577
exp_007 0.0093 0.0932    10.021505
exp_009 0.0092 0.0936    10.173913
exp_012 0.0090 0.0913    10.144444
exp_024 0.0087 0.0893    10.264368
exp_026 0.0085 0.0887    10.435294
exp_030 0.0083 0.0877    10.566265

Mean ratio: 9.77x
Min ratio: 8.66x (exp_001)
Max ratio: 10.57x (exp_030)

=== RATIO TREND ===
exp_000: CV 0.0111, Ratio 8.85x
exp_001: CV 0.0123, Ratio 8.66x
exp_003: CV 0.0105, Ratio 9.26x
exp_005: CV 0.0104, Ratio 9.32x
exp_006: CV 0.0097, Ratio 9.75x
exp_007: CV 0.0093, Ratio 10.02x
exp_009: CV 0.0092, Ratio 10.17x
exp_012: CV 0.0090, Ratio 10.14x
exp_024: CV 0.0087, Ratio 10.26x
exp_026: CV 0.0085, Ratio 10.44x
exp_030: CV 0.0083, Ratio 10.57x


In [6]:
# Strategic decision: What should we do with our 5 remaining submissions?
print('=== STRATEGIC DECISION ===')
print(f'\nSubmissions remaining: 5')
print(f'Target: {target}')
print(f'Best LB: 0.0877')
print(f'Gap to target: {0.0877 / target:.2f}x')

print(f'\n=== OPTIONS ===')
print(f'\n1. Submit exp_043 (aggressive regularization)')
print(f'   - CV: {exp_043_cv:.6f}')
print(f'   - Predicted LB: {predicted_lb_043:.4f}')
print(f'   - Purpose: Test overfitting hypothesis')
print(f'   - Risk: LB might be worse than best')
print(f'   - Reward: If hypothesis confirmed, we know to pursue more regularization')

print(f'\n2. Submit best CV model (exp_032)')
print(f'   - CV: 0.008194')
print(f'   - Predicted LB: {slope * 0.008194 + intercept:.4f}')
print(f'   - Purpose: Establish new best LB baseline')
print(f'   - Risk: Might not improve LB much')
print(f'   - Reward: Might get new best LB')

print(f'\n3. Try a completely different approach first')
print(f'   - E.g., pure GP model, different features, domain adaptation')
print(f'   - Purpose: Break out of current CV-LB relationship')
print(f'   - Risk: Might be worse')
print(f'   - Reward: Might find a fundamentally better approach')

print(f'\n=== RECOMMENDATION ===')
print(f'Submit exp_043 to test the overfitting hypothesis.')
print(f'This is the highest-leverage action because:')
print(f'1. It directly tests whether regularization can reduce the CV-LB gap')
print(f'2. The result will inform all future experiments')
print(f'3. Even if LB is worse, we learn something valuable')
print(f'4. We have 5 submissions - can afford to test')

=== STRATEGIC DECISION ===

Submissions remaining: 5
Target: 0.0347
Best LB: 0.0877
Gap to target: 2.53x

=== OPTIONS ===

1. Submit exp_043 (aggressive regularization)
   - CV: 0.009002
   - Predicted LB: 0.0912
   - Purpose: Test overfitting hypothesis
   - Risk: LB might be worse than best
   - Reward: If hypothesis confirmed, we know to pursue more regularization

2. Submit best CV model (exp_032)
   - CV: 0.008194
   - Predicted LB: 0.0877
   - Purpose: Establish new best LB baseline
   - Risk: Might not improve LB much
   - Reward: Might get new best LB

3. Try a completely different approach first
   - E.g., pure GP model, different features, domain adaptation
   - Purpose: Break out of current CV-LB relationship
   - Risk: Might be worse
   - Reward: Might find a fundamentally better approach

=== RECOMMENDATION ===
Submit exp_043 to test the overfitting hypothesis.
This is the highest-leverage action because:
1. It directly tests whether regularization can reduce the CV-LB gap